In [16]:
!python3 --version

Python 3.8.10


In [21]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
import os

In [26]:
def subgraph2vec(spark_session, data_file_path=None):
    schema = StructType([
        StructField("source", IntegerType(), True),
        StructField("target", IntegerType(), True),
        StructField("edge_type", StringType(), True)
    ])

    data_file_path = "/opt/spark/data/large_graph.txt"
    
    df = spark_session.read.option("delimiter", "|").option("header", "false") \
        .schema(schema).csv(data_file_path)
    
    total_records = df.count()
    print(f"📋 Total edges processed: {total_records:,}")
    
    return df

In [27]:

spark = None
try:
    spark = SparkSession.builder \
        .appName("SimpleGraphProcessing_v3") \
        .master("spark://spark-master:7077") \
        .config("spark.sql.adaptive.enabled", "true") \
        .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
        .getOrCreate()
    graph_df = subgraph2vec(spark)
    print("✅ Graph processing completed successfully!")
except Exception as e:
    print(f"❌ Failed to start Spark or process graph: {e}")
    print(f"Error type: {type(e).__name__}")
    import traceback
    traceback.print_exc()    
finally:
    if spark:
        spark.stop()
        print("\n🔄 Spark session stopped")

📋 Total edges processed: 100,001
✅ Graph processing completed successfully!

🔄 Spark session stopped

🔄 Spark session stopped
